# Gender diversity in pop music charts - II - Female artist recommender through ML

## Import Libraries and Dataframes from part I

In [15]:
import numpy as np
import pandas as pd

import requests
import re
import json
import csv
import os
from dotenv import load_dotenv
import ast

import pymysql.cursors
from sqlalchemy import create_engine, MetaData
from sqlalchemy.schema import CreateTable
from sqlalchemy import text
import getpass
import pickle

import time
from datetime import datetime, timedelta
import locale

from bs4 import BeautifulSoup

import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from scipy.stats import logistic

from tqdm.notebook import tqdm

import musicbrainzngs

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.neighbors import NearestNeighbors

import random

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from IPython.display import IFrame

### Import .env
for Musicbrainz useragent & Last.fm API key

In [16]:
load_dotenv('.env')

True

### Reimport Dataframes pickle backups

In [17]:
#charts_global = pd.read_csv('charts_global.csv', sep=',') # CSV messes up our dataypes

artist_info_with_bio = pd.read_pickle("artist_info_with_bio.pkl") 
charts_global = pd.read_pickle("charts_global.pkl")

In [4]:
charts_global

,chart_id,chart_position,performer,song,song_id,chart_date,main_artist,artist_featured,chart_url,previous_week,chart_name,instance,time_on_chart,consecutive_weeks,peak_position,worst_position,chart_debut
0,0,1,Peter et Sloane,"Besoin de rien, envie de toi","besoin de rien, envie de toi-peter et sloane",1984-10-26,Peter et Sloane,,https://www.chartsinfrance.net/charts/8444/sin...,NaN,Top50 FR,NaN,NaN,NaN,NaN,NaN,NaT
1,1,2,Cookie Dingler,Femme libérée,femme libérée-cookie dingler,1984-10-26,Cookie Dingler,,https://www.chartsinfrance.net/charts/8444/sin...,NaN,Top50 FR,NaN,NaN,NaN,NaN,NaN,NaT
2,2,3,Scorpions,Still Loving You,still loving you-scorpions,1984-10-26,Scorpions,,https://www.chartsinfrance.net/charts/8444/sin...,NaN,Top50 FR,NaN,NaN,NaN,NaN,NaN,NaT
3,3,4,Chris De Burgh,High On Emotion,high on emotion-chris de burgh,1984-10-26,Chris De Burgh,,https://www.chartsinfrance.net/charts/8444/sin...,NaN,Top50 FR,NaN,NaN,NaN,NaN,NaN,NaT
4,4,5,Stevie Wonder,I Just Called To Say I Love You,i just called to say i love you-stevie wonder,1984-10-26,Stevie Wonder,,https://www.chartsinfrance.net/charts/8444/sin...,NaN,Top50 FR,NaN,NaN,NaN,NaN,NaN,NaT
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
488840,488840,99,Alan Jackson,www.memory,www.memory-alan jackson,2001-02-10,Alan Jackson,,https://www.billboard.com/charts/hot-100/2001-...,89.0,Billboard Hot100,1.0,15.0,14.0,45.0,99.0,2000-11-04
488841,488841,66,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),¿dònde està santa claus? (where is santa claus...,1958-12-13,Augie Rios,,https://www.billboard.com/charts/hot-100/1958-...,NaN,Billboard Hot100,1.0,1.0,NaN,66.0,66.0,1958-12-13
488842,488842,53,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),¿dònde està santa claus? (where is santa claus...,1958-12-20,Augie Rios,,https://www.billboard.com/charts/hot-100/1958-...,66.0,Billboard Hot100,1.0,2.0,1.0,53.0,66.0,1958-12-13
488843,488843,47,Augie Rios,¿Dònde Està Santa Claus? (Where Is Santa Claus?),¿dònde està santa claus? (where is santa claus...,1958-12-27,Augie Rios,,https://www.billboard.com/charts/hot-100/1958-...,53.0,Billboard Hot100,1.0,3.0,2.0,47.0,66.0,1958-12-13


In [18]:
artist_info_with_bio

,Artist,MusicBrainz ID,Type,Gender,Area,Begin Date,End Date,genre_tags,other_tags,Bio
0,Boris,57652bf8-cfe8-42e7-b9a7-5572a7080d8d,Group,NaN,Japan,1992-01-01,NaT,"[ambient, doom metal, drone, drone metal, expe...","[dissonant, english, japan, japanese, raw, rep...",There are at least two different artists using...
1,Richard “Groove” Holmes,726cfe69-c905-4161-a10c-accb13d9ec26,Person,Male,United States,1931-05-02,1991-06-29,"[hard bop, soul jazz]",[jazz and blues],"Richard Arnold ""Groove"" Holmes, Born Richard A..."
2,‘Little’ Jimmy Dickens,bd7589a3-f82f-4c3d-b7e2-e57e89552da1,Person,Male,United States,1920-12-19,2015-01-02,NaN,NaN,"James Cecil Dickens (born December 19, 1920 - ..."
3,Pookie Hudson,29dc9009-015f-47c4-bd17-ed2af6d2ae0c,Person,NaN,NaN,1934-06-11,2006-01-16,NaN,NaN,"Thornton James (""Pookie"") Hudson ( born Des Mo..."
4,“Weird Al” Yankovic,7746d775-9550-4360-b8d5-c37bd448ce01,Person,Male,United States,1959-10-23,NaT,"[comedy, comedy rock, polka, pop, rock]","[accordion, american, parody, pizza, ponycore,...","Alfred Matthew ""Weird Al"" Yankovic (born Octob..."
...,...,...,...,...,...,...,...,...,...,...
10749,À la recherche de la Nouvelle Star,46acab74-83fe-4d7a-90cd-fca626f229fb,NaN,NaN,France,NaT,NaT,NaN,NaN,NaN
10750,Ärsenik,201ac6bb-c268-4e9d-9390-5305512d5ee2,Group,NaN,France,1992-01-01,NaT,[hip hop],NaN,"Ärsenik is a french rap duo, founded in 1997, ..."
10751,É o Tchan,04611209-2171-4652-98cd-e2c5e536ac04,Group,NaN,Brazil,1992-01-01,NaT,NaN,NaN,É o Tchan! (not to be confused with the ghetto...
10752,Émilie Simon,01252145-c9e8-4de5-a480-9b2bed05450a,Person,Female,France,1978-07-17,NaT,"[electronica, pop, trip hop]","[arranger, arrangeur, composer, compositeur, f...",Émilie Simon is a French singer and composer b...


## Unsupervised Machine Learning: Artist Recommender

### Feature selection

In [6]:
artist_info_with_bio

,Artist,MusicBrainz ID,Type,Gender,Area,Begin Date,End Date,genre_tags,other_tags,Bio
0,Boris,57652bf8-cfe8-42e7-b9a7-5572a7080d8d,Group,NaN,Japan,1992-01-01,NaT,"[ambient, doom metal, drone, drone metal, expe...","[dissonant, english, japan, japanese, raw, rep...",There are at least two different artists using...
1,Richard “Groove” Holmes,726cfe69-c905-4161-a10c-accb13d9ec26,Person,Male,United States,1931-05-02,1991-06-29,"[hard bop, soul jazz]",[jazz and blues],"Richard Arnold ""Groove"" Holmes, Born Richard A..."
2,‘Little’ Jimmy Dickens,bd7589a3-f82f-4c3d-b7e2-e57e89552da1,Person,Male,United States,1920-12-19,2015-01-02,NaN,NaN,"James Cecil Dickens (born December 19, 1920 - ..."
3,Pookie Hudson,29dc9009-015f-47c4-bd17-ed2af6d2ae0c,Person,NaN,NaN,1934-06-11,2006-01-16,NaN,NaN,"Thornton James (""Pookie"") Hudson ( born Des Mo..."
4,“Weird Al” Yankovic,7746d775-9550-4360-b8d5-c37bd448ce01,Person,Male,United States,1959-10-23,NaT,"[comedy, comedy rock, polka, pop, rock]","[accordion, american, parody, pizza, ponycore,...","Alfred Matthew ""Weird Al"" Yankovic (born Octob..."
...,...,...,...,...,...,...,...,...,...,...
10749,À la recherche de la Nouvelle Star,46acab74-83fe-4d7a-90cd-fca626f229fb,NaN,NaN,France,NaT,NaT,NaN,NaN,NaN
10750,Ärsenik,201ac6bb-c268-4e9d-9390-5305512d5ee2,Group,NaN,France,1992-01-01,NaT,[hip hop],NaN,"Ärsenik is a french rap duo, founded in 1997, ..."
10751,É o Tchan,04611209-2171-4652-98cd-e2c5e536ac04,Group,NaN,Brazil,1992-01-01,NaT,NaN,NaN,É o Tchan! (not to be confused with the ghetto...
10752,Émilie Simon,01252145-c9e8-4de5-a480-9b2bed05450a,Person,Female,France,1978-07-17,NaT,"[electronica, pop, trip hop]","[arranger, arrangeur, composer, compositeur, f...",Émilie Simon is a French singer and composer b...


In [19]:
gendered_artists = artist_info_with_bio[
    (artist_info_with_bio['Gender'].isin(['Male', 'Female', 'Non-binary'])) &
    artist_info_with_bio['Bio'].notna()]
# 4246 rows

In [20]:
gendered_artists = gendered_artists.reset_index(drop=True)
gendered_artists

,Artist,MusicBrainz ID,Type,Gender,Area,Begin Date,End Date,genre_tags,other_tags,Bio
0,Richard “Groove” Holmes,726cfe69-c905-4161-a10c-accb13d9ec26,Person,Male,United States,1931-05-02,1991-06-29,"[hard bop, soul jazz]",[jazz and blues],"Richard Arnold ""Groove"" Holmes, Born Richard A..."
1,‘Little’ Jimmy Dickens,bd7589a3-f82f-4c3d-b7e2-e57e89552da1,Person,Male,United States,1920-12-19,2015-01-02,NaN,NaN,"James Cecil Dickens (born December 19, 1920 - ..."
2,“Weird Al” Yankovic,7746d775-9550-4360-b8d5-c37bd448ce01,Person,Male,United States,1959-10-23,NaT,"[comedy, comedy rock, polka, pop, rock]","[accordion, american, parody, pizza, ponycore,...","Alfred Matthew ""Weird Al"" Yankovic (born Octob..."
3,A$AP Rocky,25b7b584-d952-4662-a8b9-dd8cdfbfeb64,Person,Male,United States,1988-10-03,NaT,"[east coast hip hop, hip hop]",[nigo],Rakim Athelaston Nakache Mayers (born October ...
4,Gloria Estefan,becd8cc6-a453-4183-af88-dedaaec859a6,Person,Female,United States,1957-09-01,NaT,"[dance, dance-pop, latin, latin pop, pop, pop ...","[bolero, cuban, latin freestyle]","Gloria Estefan (born Gloria María Fajardo, Sep..."
...,...,...,...,...,...,...,...,...,...,...
4241,ATB,22a096ef-c70d-4d70-ae19-4fc2412d4986,Person,Male,Nordrhein-Westfalen,1973-02-26,NaT,"[edm, electronic, trance, vocal trance]","[european, german]","André Tanneberger aka ATB (born February 26, 1..."
4242,deadmau5,4a00ec9d-c635-463a-8cd4-eb61725f0c60,Person,Male,Canada,1981-01-05,NaT,"[edm, electro, electro house, electronic, hous...",NaN,"Joel Thomas Zimmerman (born January 5, 1981), ..."
4243,k.d. lang,675c1c5e-5625-4a5e-97a2-b02aab5db2fc,Person,Female,Canada,1961-11-02,NaT,"[country, folk]",NaN,"Kathryn Dawn Lang, OC (born November 2, 1961),..."
4244,Émilie Simon,01252145-c9e8-4de5-a480-9b2bed05450a,Person,Female,France,1978-07-17,NaT,"[electronica, pop, trip hop]","[arranger, arrangeur, composer, compositeur, f...",Émilie Simon is a French singer and composer b...


In [9]:
gendered_artists.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4246 entries, 0 to 4245
Data columns (total 10 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   Artist          4246 non-null   object        
 1   MusicBrainz ID  4246 non-null   object        
 2   Type            4246 non-null   object        
 3   Gender          4246 non-null   object        
 4   Area            4183 non-null   object        
 5   Begin Date      3985 non-null   datetime64[ns]
 6   End Date        955 non-null    datetime64[ns]
 7   genre_tags      2701 non-null   object        
 8   other_tags      1677 non-null   object        
 9   Bio             4246 non-null   object        
dtypes: datetime64[ns](2), object(8)
memory usage: 331.8+ KB


In [10]:
#Exporting backup to pickle
gendered_artists.to_pickle("gendered_artists.pkl")

In [11]:
#Investigating why gendered_artists['Bio'].value_counts() returned 4 duplicated bios

duplicate_bios = gendered_artists[gendered_artists.duplicated(subset='Bio', keep=False)]

duplicate_bios

#shows that there are issues in fetching of the bios via the Last.fm API. Billie Holiday w/ John Williams bio,
# Freddy Cannon with Chubby Checker's bio; Connie Francie with Nat King Cole's.

,Artist,MusicBrainz ID,Type,Gender,Area,Begin Date,End Date,genre_tags,other_tags,Bio
477,John Williams,53b106e7-0cc6-42cc-ac95-ed8d30a3a98e,Person,Male,United States,1932-02-08,NaT,"[classical, contemporary classical, jazz, mode...","[academy award winner, american, american comp...",There are at least five artists with this name...
542,Chubby Checker,920febb2-ea05-4339-8a08-17294e60c736,Person,Male,Malvern,1941-10-03,NaT,"[rock, rock and roll]","[rhythm & blues, rock & roll]",Chubby Checker is the stage name of Ernest Eva...
781,Nat King Cole,fbe054ec-a143-4101-9e9e-64abc5ff5ac9,Person,Male,United States,1919-03-17,1965-02-15,"[easy listening, jazz, pop, r&b, swing, tradit...","[anthology, standards]","Nat ""King"" Cole (March 17th 1919–February 15th..."
794,Billie Holiday,d59c4cda-11d9-48db-8bfe-b557ee602aed,Person,Female,United States,1915-04-07,1959-07-17,"[blues, jazz, swing, traditional pop, vocal jazz]","[cotm candidate, eddie rubin, standards, vocal...",There are at least five artists with this name...
908,Connie Francis,5fcd7daf-25e1-47fd-9ba1-830b42708f84,Person,Female,United States,1937-12-12,NaT,"[brill building, pop]",[american],"Nat ""King"" Cole (March 17th 1919–February 15th..."
1544,Freddy Cannon,48c723b1-5044-4063-a934-b6a7602181b1,Person,Male,United States,1936-12-04,NaT,[rock],NaN,Chubby Checker is the stage name of Ernest Eva...
3698,Shifty,9e959825-4d85-4784-95c4-3c4b207f7a50,Person,Male,NaN,NaT,NaT,NaN,NaN,\nThere are two artists performing under the n...
3699,Shifty Shellshock,05d48096-60b8-46df-8503-811771d8abc6,Person,Male,United States,1974-08-23,NaT,NaN,NaN,\nThere are two artists performing under the n...


In [12]:
missing_values_count = gendered_artists[['Area', 'genre_tags', 'other_tags', 'Bio']].isnull().sum(axis=1)

# Get the total count of rows with missing values
total_rows_with_missing_values = len(gendered_artists[missing_values_count > 0])

# Print the result
print("Total rows with any missing values (null):", total_rows_with_missing_values)

Total rows with any missing values (null): 2857


In [13]:
missing_area_count = gendered_artists[['Area']].isnull().sum(axis=1)

# Get the total count of rows with missing values
total_rows_with_missing_area = len(gendered_artists[missing_area_count > 0])

# Print the result
print("Total rows with Area = null:", total_rows_with_missing_area)

Total rows with Area = null: 63


In [14]:
missing_area_count = artist_info_with_bio[['genre_tags']].isna().sum(axis=1)

# Get the total count of rows with missing values
total_rows_with_missing_area = len(gendered_artists[missing_area_count > 0])

# Print the result
print("Total rows with Area = na:", total_rows_with_missing_area)

Total rows with Area = na: 2029


/var/folders/xd/d3dyqcnj5nq2br9rb4b8gqx80000gn/T/ipykernel_70671/491490434.py:4: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  total_rows_with_missing_area = len(gendered_artists[missing_area_count > 0])


In [15]:
notna_count = artist_info_with_bio[['genre_tags', 'other_tags', 'Bio']].notna().sum(axis=1) #notna_count
rows_without_na = len(artist_info_with_bio[notna_count > 0])
rows_without_na

8451

In [16]:
notnull_count = artist_info_with_bio[['genre_tags', 'other_tags', 'Bio']].notna().sum(axis=1) #notna_count
rows_without_null = len(artist_info_with_bio[notnull_count > 0])
rows_without_null

8451

In [17]:
artist_info_with_bio['Area'].isnull().sum


<bound method NDFrame._add_numeric_operations.<locals>.sum of 0        False
1        False
2        False
3         True
4        False
         ...  
10749    False
10750    False
10751    False
10752    False
10753    False
Name: Area, Length: 10751, dtype: bool>

In [18]:
artist_info_with_bio[artist_info_with_bio['Gender'] == 'Non-binary']['Artist']


1102                 Miley Cyrus
1759                Lil Uzi Vert
1936    Christine and the Queens
2608                 Demi Lovato
4485               Janelle Monáe
5163                     Kehlani
8107                   Sam Smith
Name: Artist, dtype: object

In [19]:
# Filter the DataFrame based on gender and bio availability
filtered_artists = artist_info_with_bio[
    (artist_info_with_bio['Gender'].isin(['Male', 'Female', 'Non-binary'])) &
    artist_info_with_bio['Bio'].notna()
]

# Count the number of artists with genre_tags
artists_with_genre_tags = filtered_artists['genre_tags'].notna().sum()

# Count the number of artists with genre_tags and other_tags
artists_with_other_tags = filtered_artists[filtered_artists['genre_tags'].notna()]['other_tags'].notna().sum()

# Print the results
print("Number of artists with genre_tags:", artists_with_genre_tags)
print("Number of artists with genre_tags and other_tags:", artists_with_other_tags)

Number of artists with genre_tags: 2701
Number of artists with genre_tags and other_tags: 1392


### Vectorizing Dimensions

#### Vectorizing Bios with TFIDVectorizer

In [20]:
# https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html

In [21]:
bio_list = gendered_artists['Bio'].tolist()
bio_list

['Richard Arnold "Groove" Holmes, Born Richard Arnold Jackson(Camden, New Jersey, May 2, 1931 – St. Louis, Missouri, June 29, 1991) was an American jazz organist who performed in the hard bop and soul jazz genre. He is best known for his 1965 recording of "Misty," and is considered a precursor of acid jazz.\n\nHolmes burst onto the music scene in the early 1960s (his first album, on Pacific Jazz with guest Ben Webster was recorded in March 1961). An African-American, literally a heavyweight (approximately 300 pounds) and physically rotund in stature, he gained immediate respect with an inimitable style of his own. His sound was immediately recognizable in the upper register, but even more so because of his virtuosity in creating, undoubtedly, the most rapid, punctuating, and pulsating basslines of all the jazz organists. His Prestige recording of "Live at the Front Room" recorded in Newark, New Jersey is one of the great jazz performances on the Hammond-B3. He (Holmes) seemed to make f

In [22]:
vectorizer = TfidfVectorizer(stop_words="english", ngram_range=(1, 2))
vectorizer

TfidfVectorizer(ngram_range=(1, 2), stop_words='english')

In [23]:
vectors = vectorizer.fit_transform(bio_list).todense()
vectors

matrix([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]])

In [24]:
vocabulary = vectorizer.get_feature_names_out()
vocabulary

array(['00', '00 00', '00 01', ..., 'ｌｕｌｕ unlike', 'ｒａｐｈａｅｌ',
       'ｒａｐｈａｅｌ american'], dtype=object)

In [25]:
#Vectorize the bios
vectorized_bios = pd.DataFrame(vectors, columns=vocabulary, index=bio_list)
vectorized_bios

00  00 00  00 01  00 02  \
Richard Arnold "Groove" Holmes, Born Richard Ar...  0.0    0.0    0.0    0.0   
James Cecil Dickens (born December 19, 1920 - J...  0.0    0.0    0.0    0.0   
Alfred Matthew "Weird Al" Yankovic (born Octobe...  0.0    0.0    0.0    0.0   
Rakim Athelaston Nakache Mayers (born October 3...  0.0    0.0    0.0    0.0   
Gloria Estefan (born Gloria María Fajardo, Sept...  0.0    0.0    0.0    0.0   
...                                                 ...    ...    ...    ...   
André Tanneberger aka ATB (born February 26, 19...  0.0    0.0    0.0    0.0   
Joel Thomas Zimmerman (born January 5, 1981), b...  0.0    0.0    0.0    0.0   
Kathryn Dawn Lang, OC (born November 2, 1961), ...  0.0    0.0    0.0    0.0   
Émilie Simon is a French singer and composer bo...  0.0    0.0    0.0    0.0   
Born 1986 in the suburban Icelandic town of Mos...  0.0    0.0    0.0    0.0   

                                                    00 05  00 13  00 19  \
Richard Arnold "Groove" Holmes, Born Richard Ar...    0.0    0.0    0.0   
James Cecil Dickens (born December 19, 1920 - J...    0.0    0.0    0.0   
Alfred Matthew "Weird Al" Yankovic (born Octobe...    0.0    0.0    0.0   
Rakim Athelaston Nakache Mayers (born October 3...    0.0    0.0    0.0   
Gloria Estefan (born Gloria María Fajardo, Sept...    0.0    0.0    0.0   
...                                                   ...    ...    ...   
André Tanneberger aka ATB (born February 26, 19...    0.0    0.0    0.0   
Joel Thomas Zimmerman (born January 5, 1981), b...    0.0    0.0    0.0   
Kathryn Dawn Lang, OC (born November 2, 1961), ...    0.0    0.0    0.0   
Émilie Simon is a French singer and composer bo...    0.0    0.0    0.0   
Born 1986 in the suburban Icelandic town of Mos...    0.0    0.0    0.0   

                                                    00 20  00 2001  00 22  \
Richard Arnold "Groove" Holmes, Born Richard Ar...    0.0      0.0    0.0   
James Cecil Dickens (born December 19, 1920 - J...    0.0      0.0    0.0   
Alfred Matthew "Weird Al" Yankovic (born Octobe...    0.0      0.0    0.0   
Rakim Athelaston Nakache Mayers (born October 3...    0.0      0.0    0.0   
Gloria Estefan (born Gloria María Fajardo, Sept...    0.0      0.0    0.0   
...                                                   ...      ...    ...   
André Tanneberger aka ATB (born February 26, 19...    0.0      0.0    0.0   
Joel Thomas Zimmerman (born January 5, 1981), b...    0.0      0.0    0.0   
Kathryn Dawn Lang, OC (born November 2, 1961), ...    0.0      0.0    0.0   
Émilie Simon is a French singer and composer bo...    0.0      0.0    0.0   
Born 1986 in the suburban Icelandic town of Mos...    0.0      0.0    0.0   

                                                    ...   희진  희진 born   ｃ４  \
Richard Arnold "Groove" Holmes, Born Richard Ar...  ...  0.0      0.0  0.0   
James Cecil Dickens (born December 19, 1920 - J...  ...  0.0      0.0  0.0   
Alfred Matthew "Weird Al" Yankovic (born Octobe...  ...  0.0      0.0  0.0   
Rakim Athelaston Nakache Mayers (born October 3...  ...  0.0      0.0  0.0   
Gloria Estefan (born Gloria María Fajardo, Sept...  ...  0.0      0.0  0.0   
...                                                 ...  ...      ...  ...   
André Tanneberger aka ATB (born February 26, 19...  ...  0.0      0.0  0.0   
Joel Thomas Zimmerman (born January 5, 1981), b...  ...  0.0      0.0  0.0   
Kathryn Dawn Lang, OC (born November 2, 1961), ...  ...  0.0      0.0  0.0   
Émilie Simon is a French singer and composer bo...  ...  0.0      0.0  0.0   
Born 1986 in the suburban Icelandic town of Mos...  ...  0.0      0.0  0.0   

                                                    ｃ４ japanese  ｊｏｋｅｒ  \
Richard Arnold "Groove" Holmes, Born Richard Ar...          0.0    0.0   
James Cecil Dickens (born December 19, 1920 - J...          0.0    0.0   
Alfred Matthew "Weird Al" Yankovic (born Octobe...          0.0    0.0   
Rakim Athelaston Nakache Mayers (

In [26]:
vectorized_bios.shape

(4246, 793186)

#### Vectorizing Genres via CountVectorizer

In [27]:
# We need to convert the tag lists to strings to pass them into the vectorizer
genre_tags = gendered_artists['genre_tags'].apply(lambda x: ' '.join(x) if isinstance(x, list) else '')
genre_tags

0                                      hard bop soul jazz
1                                                        
2                       comedy comedy rock polka pop rock
3                              east coast hip hop hip hop
4       dance dance-pop latin latin pop pop pop rock r...
                              ...                        
4241                   edm electronic trance vocal trance
4242    edm electro electro house electronic house pro...
4243                                         country folk
4244                             electronica pop trip hop
4245      ambient contemporary classical modern classical
Name: genre_tags, Length: 4246, dtype: object

In [28]:
#https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html

#Set our vectorizer, using CountVectorizer, avoiding English stop words,
#and making pairs to better take into 2-word genres
tag_vectorizer = CountVectorizer(stop_words="english", ngram_range=(1, 2))

In [29]:
#Vectorize our tags
vectors = tag_vectorizer.fit_transform(genre_tags).todense()
vectors

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]])

In [30]:
#Get the list of tokens from the vectors
vocabulary = tag_vectorizer.get_feature_names_out()
vocabulary

array(['abstract', 'abstract hip', 'acid', ..., 'yé', 'yé yé', 'zouk'],
      dtype=object)

In [31]:
#Make a Dataframe of the vectors
vectorized_genres = pd.DataFrame(vectors, columns=vocabulary, index=genre_tags)
vectorized_genres

,abstract,abstract hip,acid,acid jazz,acid rock,acid techno,acoustic,acoustic blues,acoustic rock,afro,...,world,world fusion,yacht,yacht rock,yodeling,yodeling nashville,yodeling traditional,yé,yé yé,zouk
genre_tags,,,,,,,,,,,,,,,,,,,,,
hard bop soul jazz,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
comedy comedy rock polka pop rock,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
east coast hip hop hip hop,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
dance dance-pop latin latin pop pop pop rock rock son cubano,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
edm electronic trance vocal trance,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
edm electro electro house electronic house progressive house,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
country folk,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [32]:
vectorized_genres.shape

(4246, 1916)

#### Vectorizing Other_tags via CountVectorizer

In [33]:
# Convert the tag lists to strings to pass them into the vectorizer
other_tags = gendered_artists['other_tags'].apply(lambda x: ' '.join(x) if isinstance(x, list) else '')
other_tags

0                                          jazz and blues
1                                                        
2           accordion american parody pizza ponycore star
3                                                    nigo
4                            bolero cuban latin freestyle
                              ...                        
4241                                      european german
4242                                                     
4243                                                     
4244    arranger arrangeur composer compositeur female...
4245                                   composer icelandic
Name: other_tags, Length: 4246, dtype: object

In [34]:
#Vectorize the tags using TFID, with tag_vectorizer already defined above (for genres)
vectors = tag_vectorizer.fit_transform(other_tags).todense()

In [35]:
#Get the tokens from the vectors
vocabulary = tag_vectorizer.get_feature_names_out()

In [36]:
#Make a Dataframe of the vectors
vectorized_tags = pd.DataFrame(vectors, columns=vocabulary, index=other_tags)
vectorized_tags

,00,00 80,00289,00289 479,0056,0056 19th,00s,00s 10s,00s 80s,1036,...,york passionate,york pennsylvania,york poetic,zealand,الطلياني,رشيد,رشيد طه,رضى,رضى الطلياني,طه
other_tags,,,,,,,,,,,,,,,,,,,,,
jazz and blues,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
accordion american parody pizza ponycore star,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
nigo,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
bolero cuban latin freestyle,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
european german,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Vectorizing Areas via CountVectorizer? (maybe later)


### Optional PCA

### Normalizing

#### Concatenate dimensions

In [37]:
#Resetting indexes of our vectors dataframes before concat

vectorized_genres = vectorized_genres.reset_index(drop=True)
vectorized_tags = vectorized_tags.reset_index(drop=True)
vectorized_bios = vectorized_bios.reset_index(drop=True)

vectorized_dimensions = pd.concat([vectorized_genres, vectorized_tags, vectorized_bios], axis=1)
vectorized_dimensions

,abstract,abstract hip,acid,acid jazz,acid rock,acid techno,acoustic,acoustic blues,acoustic rock,afro,...,희진,희진 born,ｃ４,ｃ４ japanese,ｊｏｋｅｒ,ｊｏｋｅｒ visual,ｌｕｌｕ,ｌｕｌｕ unlike,ｒａｐｈａｅｌ,ｒａｐｈａｅｌ american
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4242,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4243,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4244,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
#Export backup to pickles
#vectorized_dimensions.to_pickle("vectorized_dimensions.pkl")

In [39]:
vectorized_dimensions_sample = vectorized_dimensions.head()
row_sum = vectorized_dimensions_sample.sum(axis=1)
row_sum

0    26.462772
1    18.530686
2    41.943876
3    20.532671
4    56.946373
dtype: float64

In [40]:
weighted_dimensions = pd.concat([vectorized_genres * 5, vectorized_tags * 2, vectorized_bios], axis=1)
weighted_dimensions

,abstract,abstract hip,acid,acid jazz,acid rock,acid techno,acoustic,acoustic blues,acoustic rock,afro,...,희진,희진 born,ｃ４,ｃ４ japanese,ｊｏｋｅｒ,ｊｏｋｅｒ visual,ｌｕｌｕ,ｌｕｌｕ unlike,ｒａｐｈａｅｌ,ｒａｐｈａｅｌ american
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4242,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4243,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4244,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [41]:
# Export weighted dimensions to pickles for backup
# weighted_dimensions.to_pickle("weighted_dimensions.pkl")

In [ ]:
weighted_dimensions_sample = weighted_dimensions.head()
row_sum = weighted_dimensions_sample.sum(axis=1)
row_sum

In [4]:
# Import weighted dimensions from pickles backup
# weighted_dimensions.to_pickle("weighted_dimensions.pkl")
weighted_dimensions = pd.read_pickle("weighted_dimensions.pkl") 
weighted_dimensions

,abstract,abstract hip,acid,acid jazz,acid rock,acid techno,acoustic,acoustic blues,acoustic rock,afro,...,희진,희진 born,ｃ４,ｃ４ japanese,ｊｏｋｅｒ,ｊｏｋｅｒ visual,ｌｕｌｕ,ｌｕｌｕ unlike,ｒａｐｈａｅｌ,ｒａｐｈａｅｌ american
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4242,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4243,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4244,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### Normalize using Normalizer

In [6]:
# https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Normalizer.html

normalizer = Normalizer(norm='l2')
weighted_dimensions_normal = normalizer.fit_transform(weighted_dimensions)


In [7]:
weighted_dimensions_normal = pd.DataFrame(weighted_dimensions)
weighted_dimensions_normal

,abstract,abstract hip,acid,acid jazz,acid rock,acid techno,acoustic,acoustic blues,acoustic rock,afro,...,희진,희진 born,ｃ４,ｃ４ japanese,ｊｏｋｅｒ,ｊｏｋｅｒ visual,ｌｕｌｕ,ｌｕｌｕ unlike,ｒａｐｈａｅｌ,ｒａｐｈａｅｌ american
0,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4241,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4242,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4243,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4244,0,0,0,0,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
weighted_dimensions_normal_sample = weighted_dimensions_normal.head(5)
row_sum = weighted_dimensions_normal_sample.sum(axis=1)
row_sum

0     57.462772
1     18.530686
2     96.943876
3     65.532671
4    155.946373
dtype: float64

In [13]:
#Export to pickles for backup
#weighted_dimensions_normal.to_pickle("weighted_dimensions_normal.pkl")

### Nearest Neighbors

https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.NearestNeighbors.html

In [29]:
# Train the Nearest Neighbor Model
neighbor = NearestNeighbors(n_neighbors=5)
neighbor.fit(weighted_dimensions_normal)

NearestNeighbors()

In [39]:
# Save the model using pickle
with open('nearest_neighbors_model.pkl', 'wb') as f:
    pickle.dump(neighbor, f)

## User Input for recommendation

### Set up

In [22]:
#Merge charts data
merged_data = gendered_artists.merge(charts_global, left_on='Artist', right_on='main_artist', how='left')
merged_data = merged_data.dropna(subset=['chart_position'])

# Group the DataFrame by "Artist" and "song", and create a new column with the count of occurrences
merged_data['count'] = merged_data.groupby(['Artist', 'song'])['song'].transform('size')

# Sort the DataFrame by "Artist", count of occurrences in descending order, and chart date in chronological order
merged_data = merged_data.sort_values(['Artist', 'count', 'chart_date'], ascending=[True, False, True])

# Drop duplicates of "Artist" to keep only the rows with the highest count of occurrences and earliest "chart_date" in case of ties
merged_data = merged_data.drop_duplicates('Artist', keep='first')

# Select the desired columns
merged_data = merged_data[['Artist', 'Type', 'Gender', 'Area', 'Begin Date', 'End Date', 'genre_tags', 'other_tags', 'Bio', 'performer', 'song', 'main_artist', 'chart_date', 'chart_name']]

# Reset the index
merged_data = merged_data.reset_index(drop=True)

merged_data

,Artist,Type,Gender,Area,Begin Date,End Date,genre_tags,other_tags,Bio,performer,song,main_artist,chart_date,chart_name
0,12 Gauge,Person,Male,United States,1968-01-01,NaT,[hip hop],NaN,There are at least 2 artist with this name:\n\...,12 Gauge,Dunkie Butt (Please Please Please),12 Gauge,1994-01-08,Billboard Hot100
1,2 Chainz,Person,Male,United States,1976-09-10,NaT,"[dirty south, hardcore hip hop, hip hop, south...","[2010s, criminal]","Tauheed Epps (born September 12, 1976), better...",2 Chainz Featuring Drake,No Lie,2 Chainz,2012-05-26,Billboard Hot100
2,2 Pistols,Person,Male,United States,1983-06-11,NaT,NaN,NaN,Over the Past 5 years 2 Pistols has become a r...,2 Pistols Featuring T-Pain & Tay Dizm,She Got It,2 Pistols,2008-03-01,Billboard Hot100
3,2Pac,Person,Male,United States,1971-06-16,1996-09-13,"[conscious hip hop, g-funk, gangsta rap, hip h...","[1990s, 90s, death by murder]","Tupac Amaru Shakur (born Lesane Parish Crooks,...",2Pac,I Get Around,2Pac,1993-07-03,Billboard Hot100
4,50 Cent,Person,Male,United States,1975-07-06,NaT,"[east coast hip hop, gangsta rap, hardcore hip...","[2000s, criminal, rap]","Curtis James Jackson III (born July 6, 1975 in...",50 Cent,In Da Club,50 Cent,2003-01-11,Billboard Hot100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3496,will.i.am,Person,Male,United States,1975-03-15,NaT,"[hip hop, pop rap]",[pop and chart],"William James Adams Jr. (born March 15, 1975),...",will.i.am,This Is Love,will.i.am,2012-06-22,Top100 FR
3497,Élodie Frégé,Person,Female,France,1982-02-15,NaT,[pop],"[french, world]",Élodie Frégé won France's Star Academy in 200...,Élodie Frégé,Je te dis non,Élodie Frégé,2004-11-19,Top100 FR
3498,Émilie Simon,Person,Female,France,1978-07-17,NaT,"[electronica, pop, trip hop]","[arranger, arrangeur, composer, compositeur, f...",Émilie Simon is a French singer and composer b...,Émilie Simon,Désert,Émilie Simon,2014-12-05,Top100 FR
3499,Étienne Daho,Person,Male,France,1956-01-14,NaT,"[pop, pop rock, synth-pop]",NaN,"Étienne Daho (born January 14, 1956 in Oran, A...",Étienne Daho,Le premier jour,Étienne Daho,1998-03-13,Top100 FR


In [236]:
#Checking the most common genres in our list:
#artist_genre_table['genre_id'].value_counts().head(20)

1267    1075
1387    1034
786      793
1523     647
1275     584
50       336
1343     334
854      320
374      317
381      297
544      290
755      276
1485     250
428      249
610      188
1143     176
815      176
642      169
1510     166
1276     155
Name: genre_id, dtype: int64

In [237]:
#genre_indices = [1267, 1387, 786, 1523, 1275, 50, 1343, 854, 374, 381, 544, 755, 1485, 428, 610, 1143, 815, 642, 1510, 1276]
#selected_genres = genres_df.loc[genre_indices, 'genre']
#print(selected_genres)

1267                  pop
1387                 rock
786               hip hop
1523                 soul
1275             pop rock
50       alternative rock
1343                  r&b
854                  jazz
374      contemporary r&b
381               country
544            electronic
755             hard rock
1485    singer-songwriter
428             dance-pop
610                  folk
1143             new wave
815            indie rock
642                  funk
1510            soft rock
1276             pop soul
Name: genre, dtype: object


### Get User input

In [23]:
# Define the genres dictionary
most_common_genres = {
    1: 'Pop \U0001F483',
    2: 'Rock \U0001F3B8',
    3: 'Hip hop \U0001F3A4',
    4: 'R&B \U0001F48E',
    5: 'Electronic \U0001F47E',
    6: 'Jazz \U0001F3B7',
    7: 'Folk \U0001FA95',
    8: 'Soul \U0001F399',
    9: 'Alternative Rock \U0001F918'
}

# Filter our data for FR / US charts:
fr_artists = merged_data[merged_data['chart_name'].str.contains('Top')]
fr_artists = fr_artists.reset_index(drop=True)
us_artists = merged_data[merged_data['chart_name'].str.contains('Billboard')]
us_artists = us_artists.reset_index(drop=True)  # Reset the index

# Get user input for country
country_input = input("Let's find an artist you like from the charts history. Enter 'F' if you prefer French charts or 'U' for US charts: ").upper()

if country_input == "F":
    # Select random artist from merge_data where chart_name contains "Top"
    selected_artists = fr_artists
elif country_input == "U":
    selected_artists = us_artists
else:
    print("Invalid input. Please try again.")

if selected_artists is not None:
    # Get user input for genre
    print("\U0001F3B5  \U0001F3B6  \U0001F3B5")
    print("  \U0001F3A7   ")
    print("Select a genre: ")
    print("\U0001F3B5  \U0001F3B6  \U0001F3B5")

    for key, value in most_common_genres.items():
        print(f"{key}. {value}")

    print("\U0001F3B5  \U0001F3B6  \U0001F3B5")

    genre_input = input("Enter the genre number: ")

    try:
        genre_number = int(genre_input)
        if genre_number in most_common_genres:
            selected_genre = most_common_genres[genre_number].lower()
            # split on first whitespace from the right to remove the emoji
            selected_genre = selected_genre.rsplit(' ', 1)[0]

            #selected_genre = selected_genre.split('\U0001F')[0]
            #selected_genre = selected_genre.strip()

            # Filter artists by selected genre
            genre_filtered_artists = selected_artists[selected_artists['genre_tags'].notna() & selected_artists['genre_tags'].apply(lambda x: selected_genre in x if isinstance(x, list) else False)]
            genre_filtered_artists = genre_filtered_artists.reset_index(drop=True)

            # Check if there are artists available for the selected genre
            if genre_filtered_artists.empty:
                print("No artists found for the selected genre.")
            else:
                while True:
                    # Select random artist
                    random_artist = random.choice(genre_filtered_artists['Artist'])
                    print(">> Do you want us to recommend an artist similar to >>", random_artist)

                    # Ask user if they want to proceed with the recommendation based on that first artist
                    proceed_to_reco = input("Press Y for Yes or N for another artist: ").upper()

                    if proceed_to_reco == 'Y':
                        print("Okay, let our recommender find a similar artist to", random_artist)
                        break  # Exit the loop if user wants to proceed with a similar artist
                    elif proceed_to_reco == 'N':
                        print("Let's select another artist.\n")
                    else:
                        print("Invalid input \U0001F62D Please enter 'Y' or 'N'.")
        else:
            print("Invalid genre number \U0001F62D Please try again.")
    except ValueError:
        print("Invalid input \U0001F62D Please enter a digit for the genre number.")
else:
    print("No artists found for the selected country \U0001F62D")

🎵  🎶  🎵
  🎧   
Select a genre: 
🎵  🎶  🎵
1. Pop 💃
2. Rock 🎸
3. Hip hop 🎤
4. R&B 💎
5. Electronic 👾
6. Jazz 🎷
7. Folk 🪕
8. Soul 🎙
9. Alternative Rock 🤘
🎵  🎶  🎵
>> Do you want us to recommend an artist similar to >> Alanis Morissette
Okay, let our recommender find a similar artist to Alanis Morissette


## Get Nearest Neighbor Female artist to recommend

## Play a track from recommended artist

In [25]:
#Find the index of random_artist in gendered_artist
random_artist_index = gendered_artists.index[gendered_artists['Artist'] == random_artist][0]
random_artist_index

90

In [28]:
gendered_artists.iloc[90]

Artist                                            Alanis Morissette
MusicBrainz ID                 4bdcee62-4902-4773-8cd1-e252e2e31225
Type                                                         Person
Gender                                                       Female
Area                                                         Canada
Begin Date                                      1974-06-01 00:00:00
End Date                                                        NaT
genre_tags        [alternative pop, alternative rock, folk rock,...
other_tags        [1990s, adult alternative, englisch, english, ...
Bio               Alanis Nadine Morissette (born June 1, 1974) i...
Name: 90, dtype: object

In [31]:
#Find the nearest neighbor
indices = neighbor.kneighbors(weighted_dimensions_normal.iloc[random_artist_index:random_artist_index+1])[1]
indices


array([[  90, 1335,  816, 3724, 3818]])

In [32]:
#Iterate through the neighbors to return the Female artists
female_neighbors = []
for index in indices[0]:
    if gendered_artists.loc[index, 'Gender'] == 'Female':
        female_neighbors.append(index)
closest_female_neighbors = gendered_artists.loc[female_neighbors]


In [33]:
female_neighbors

[90, 3724, 3818]

In [34]:
gendered_artists.iloc[3724]

Artist                                              Sinéad O’Connor
MusicBrainz ID                 c78a77fa-507c-4c07-947a-0355029453bd
Type                                                         Person
Gender                                                       Female
Area                                                        Ireland
Begin Date                                      1966-12-08 00:00:00
End Date                                                        NaT
genre_tags        [alternative rock, celtic rock, folk, pop, pop...
other_tags        [alternative, ireland, irish, irlandais, irlan...
Bio               Sinéad Marie Bernadette O'Connor (/ʃɪˈneɪd oʊˈ...
Name: 3724, dtype: object

In [35]:
gendered_artists.iloc[3818]

Artist                                                 Stevie Nicks
MusicBrainz ID                 b7f2cca2-72c6-41fb-ae33-53370fc62fe7
Type                                                         Person
Gender                                                       Female
Area                                                  United States
Begin Date                                      1948-05-26 00:00:00
End Date                                                        NaT
genre_tags        [country rock, pop, pop rock, rock, singer-son...
other_tags                                                      NaN
Bio               Stephanie Lynn Nicks (born May 26, 1948)[1] is...
Name: 3818, dtype: object

In [97]:
#Let's update recommended_artist below once we have our NEAREST NEIGHBOR (ALMOST THERE)

recommended_artist = random_artist

# Filter merged_data for the randomly selected artist
artist_data = merged_data[merged_data['Artist'] == random_artist]

# Get the recommended song for the selected artist
recommended_song = artist_data['song'].values
recommended_song = str(recommended_song[0])

# Print the recommended song
recommended_song

"This Time I Know It's For Real"

In [94]:
#Connect to the Spotify API
c_id = os.environ['SPOTIFY_USER_ID']
c_se = os.environ['SPOTIFY_SECRET']

sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=c_id, client_secret=c_se))

In [79]:
# embedded player
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
                  width="320",
                  height="80",
                  frameborder="0",
                  allowtransparency="true",
                  allow="encrypted-media")

In [98]:
# Search for tracks by the artist and track name
#q = recommended_song ' - ' recommended_song
results = sp.search(q=f'artist:{recommended_artist} track:{recommended_song}', type='track', limit=10)

if not results["tracks"]["items"]:
        raise ValueError(f"No match found for {recommended_song} by {recommended_artist}")

# if multiple matches found, ask the user to choose the correct artist
if len(results["tracks"]["items"]) > 1:
    print("Multiple matches found. Please pick the wanted song and artist:")
    for i, item in enumerate(results["tracks"]["items"], 1):
        print(f"{i}. {item['artists'][0]['name']}")
    
    artist_choice = int(input("Enter the number for the correct artist: ")) - 1
    track_id = results["tracks"]["items"][artist_choice]["id"]

else:
    # extract track ID of the first result
    track_id = results["tracks"]["items"][0]["id"]


# Play the chosen song in the music embedded player
display(play_song(track_id))

Multiple matches found. Please pick the wanted song and artist:
1. Donna Summer
2. Donna Summer
3. Donna Summer
4. Donna Summer
5. Donna Summer
6. Donna Summer
7. Donna Summer
8. Donna Summer
9. Donna Summer
10. Donna Summer


In [ ]:
#Print Artist name + Area + Begin Date + End Date + Genre_tags + Bio

# Thank you!